In [2]:
import sys
!{sys.executable} -m pip install librosa

import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
from scipy import signal
from scipy.fft import fftshift
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter,filtfilt
import librosa
import librosa.display

import torch
import torchvision
from torch.autograd import Variable
from torchvision.io import read_image
import torch.nn.functional as F
import matplotlib.pyplot as plt

    100% |################################| 215kB 1.9MB/s ta 0:00:011
    100% |################################| 378kB 1.3MB/s eta 0:00:01
    100% |################################| 327kB 1.5MB/s eta 0:00:01
    100% |################################| 61kB 3.9MB/s eta 0:00:01
  Running setup.py bdist_wheel for audioread ... done
  Stored in directory: /root/.cache/pip/wheels/6d/55/fc/0d9cb2a5a52cc24d86cfcd5db4d40f448a43b9a6046c019fc1
  Running setup.py bdist_wheel for resampy ... done
  Stored in directory: /root/.cache/pip/wheels/fa/c1/56/e0e12c6f7f3d2cdea9712b35136a2d40a7817c6210ec096485
Successfully built audioread resampy
